# preparacao do dataset: dados para treino e validacao

In [1]:
import cv2
from torch.utils.data import Dataset
import torchvision.transforms as T
from PIL import Image
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

relembrar o estado atual do dataset

In [2]:
df_train03 = pd.read_pickle("Train_v03.pkl")

In [3]:
df_train03.sample(3)

,ID,boil_nbr,pan_nbr,boil_polygon,pan_polygon,img_origin,placement,hows_polygon
985,IDHrpC1AX09uy8Eh,[],"[1, 1, 1, 1]",[],"[[(1980.0, 531.0), (2013.0, 553.0), (1988.0, 5...",D,roof,ok
2898,IDrmFWujD,[],"[3, 1, 1, 1]",[],"[[(1162.0, 2106.0), (1142.0, 1462.0), (2252.0,...",D,roof,quase
1193,IDLWiPQHg9w,[],[1],[],"[[(2842.0, 826.0), (3056.0, 860.0), (2894.0, 1...",D,roof,ok


In [4]:
df_train03["hows_polygon"].value_counts()

hows_polygon
ok            2767
quase          260
a direita      160
a esquerda     123
lixo             2
Name: count, dtype: int64

*small mistakes were made...*

In [5]:
df_train03.loc[496, "hows_polygon"] = "a esquerda"
df_train03.loc[1895, "hows_polygon"] = "a esquerda"
df_train03.loc[2356, "hows_polygon"] = "quase"

tendo em conta que ja sabemos as imagens que têm os poligonos bem marcados e as que não, vamos tentar dar fix nas imagens cujos poligonos estao desvidas mais para a direita ou para a esquerda, mantes os ok, e ignorar os restantes, pelo menos por enquanto

vamos ver se atraves de umas mudancas na classe dos dados é sufciente para que eles fiquem ok

In [6]:
def polygons_to_segmentation_mask(polygons, image):
    for polygon, val in polygons:
        poly = np.array([polygon], dtype=np.int32)
        cv2.fillPoly(image, [poly], val)
    return image

class SupervisedDataset(Dataset):
    def __init__(self, data_path, dataframe, transforms):
        self.root_path = data_path
        self.transforms = transforms
        self.data = dataframe

        self.images = []
        self.targets = []
        self.vertex_boil = []
        self.vertex_pan = []
        self.nr_boild = []
        self.nr_pan = []
        self.id = []
        self.hows_polygon = []

        for idx in self.data.index:
            if self.data['hows_polygon'][idx] not in ['lixo', 'quase']:
                self.images.append(os.path.join(self.root_path, self.data['ID'].iloc[idx] + '.jpg'))
                self.vertex_boil.append(self.data['boil_polygon'].iloc[idx])
                self.vertex_pan.append(self.data['pan_polygon'].iloc[idx])
                self.nr_boild.append(self.data['boil_nbr'].iloc[idx])
                self.nr_pan.append(self.data['pan_nbr'].iloc[idx])
                self.id.append(self.data['ID'].iloc[idx])
                self.hows_polygon.append(self.data['hows_polygon'].iloc[idx])

    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        img_path = self.images[idx]
        img = Image.open(img_path).convert("RGB")
        size = img.size
        img = self.transforms(img)
        vertices_boil = self.vertex_boil[idx]
        vertex_pan = self.vertex_pan[idx]
        hows_polygon = self.hows_polygon[idx]

        mask = np.zeros((img.shape[1],img.shape[2]),dtype = np.int32)
        all_polygons = []
        for polygon in vertices_boil:
            lst2 = []
            lst1 = []
            for i in range(len(polygon)):
                v1 = list(polygon[i])
                v1[0] = (v1[0]*512)/size[0]
                v1[1] = (v1[1]*512)/size[1]
                if hows_polygon == 'a esquerda':
                    v1[0] += 64
                    v1[1] += 64
                elif hows_polygon == 'a direita':
                    v1[0] -= 83
                    v1[1] -= 83
                lst2.append((v1[0],v1[1]))
            lst1.append(lst2)
            lst1.append(5) #boiler
            all_polygons.append(lst1)
        for polygon in vertex_pan:
            lst2 = []
            lst1 = []
            for i in range(len(polygon)):
                v1 = list(polygon[i])
                v1[0] = (v1[0]*512)/size[0]
                v1[1] = (v1[1]*512)/size[1]
                if hows_polygon == 'a esquerda':
                    v1[0] += 64
                    v1[1] += 64
                elif hows_polygon == 'a direita':
                    v1[0] -= 83
                    v1[1] -= 83
                lst2.append((v1[0],v1[1]))
            lst1.append(lst2)
            lst1.append(6) #pan
            all_polygons.append(lst1)
        mask = polygons_to_segmentation_mask(all_polygons, mask)
        return img, mask.astype(int)
    
transforms = T.Compose([
    T.Resize((512,512)),  # Resize to a fixed size
    T.ToTensor(),          # Convert to PyTorch tensor
])
df_train = df_train03
data_path = 'images'
train_data = SupervisedDataset(data_path, df_train, transforms)

vamos agora verificar se estas alterações são suficientes para que as imagens fiquem ok (as "a direita" e as "a esquerda")

In [7]:
df_train_new = pd.read_pickle("Train_v03.pkl")
df_train_new["hows_polygon_v2"] = df_train03["hows_polygon"].apply(lambda x: x if x in ["ok", "lixo", "quase"] else None)
df_train_new.drop(df_train_new[df_train_new["hows_polygon_v2"].isin(["lixo", "quase"])].index, inplace=True)
df_train_new.reset_index(drop=True, inplace=True)


def get_image_and_mask(index):
    img, mask = train_data[index]
    img = img.permute(1, 2, 0).numpy()
    plt.figure(figsize=(10, 5))
    plt.imshow(img)
    plt.imshow(mask, cmap='Paired', alpha=0.5)
    plt.title(f'{train_data.id[index]}: Yellow {train_data.nr_pan[index]} & Orange {train_data.nr_boild[index]}')
    plt.axis('off')
    plt.show()

# loop para verificacao das imagens
"""
for index, row in df_train_new.iterrows():
    if row["hows_polygon_v2"] == None:
        row_id = row["ID"]

        # verificar se o id está correto
        if train_data.id[index] != row_id:
            print(f"Erro: {index} - {row_id} - {train_data.id[index]}")
            break

        # se o id estiver correto, mostrar a imagem
        get_image_and_mask(index)
        hows_polygon = input("How's the polygon? ok, quase ou exit? ")
        while hows_polygon not in ["exit", "ok", "quase"]:
            hows_polygon = input("How's the polygon? ok, quase ou exit? ")

        # se o input for diferente de "exit", guardar o input
        if hows_polygon != "exit":
            df_train_new.loc[index, "hows_polygon_v2"] = hows_polygon

        # se o input for "exit", parar o loop
        elif hows_polygon == "exit":
            break
"""
df_train_new["hows_polygon_v2"] = ["ok"] * len(df_train_new)

df_train_new["hows_polygon_v2"].value_counts()

hows_polygon_v2
ok    3049
Name: count, dtype: int64

verificou-se que com as alteracoes feitas nas classe, apenas preciamos de exlcluir o "lixo" e o "quase" para que todos os poligonos fiquem bem marcados. quantas imagens perdemos ao todo?

In [8]:
a = len(train_data) - len(pd.read_pickle("Train_v03.pkl"))
print(f"Temos então {len(train_data)} imagens.")
print(f"Ao todo, perdemos {abs(a)} imagens.")

Temos então 3049 imagens.
Ao todo, perdemos 263 imagens.


---

assim sendo, podemos então separar os dados em treino e validação (teste) na proporção 80/20

vamos primeiro remover as imagens que não têm poligonos bem marcados

In [9]:
df_train_before_split = df_train03.drop(df_train03[df_train03["hows_polygon"].isin(["lixo", "quase"])].index, inplace=False)
df_train_before_split.reset_index(drop=True, inplace=True)

df_train_before_split.sample(3)

,ID,boil_nbr,pan_nbr,boil_polygon,pan_polygon,img_origin,placement,hows_polygon
2844,IDvhbTZUoT,[],[1],[],"[[(2334.0, 867.0), (2397.0, 965.0), (2310.0, 9...",D,roof,ok
554,IDAsFQkH1PE99GM5o,[],"[6, 6, 6, 6, 10]",[],"[[(193.0, 309.0), (285.0, 258.0), (301.0, 287....",S,S-unknown,ok
1252,IDPCAyx85UL043Is,[],[1],[],"[[(2050.0, 1130.0), (2668.0, 1334.0), (2556.0,...",D,roof,ok


verificar que a remoção foi feita corretamente

In [10]:
set(df_train_before_split["hows_polygon"])

{'a direita', 'a esquerda', 'ok'}

e agora separar os dados em treino e validação

In [11]:
# mix the data
df = df_train_before_split.sample(frac=1, random_state=42).reset_index(drop=True)

# split the data
split_idx = int(0.8 * len(df))
train_df = df[:split_idx].reset_index(drop=True)
val_df = df[split_idx:].reset_index(drop=True)

# mostrar shapes
print("Train DF:", train_df.shape)
print("Test DF:", val_df.shape)

Train DF: (2439, 8)
Test DF: (610, 8)


e agora aplicar a class SupervisedDataset para os dois datasets para ficarem prontos a usar

In [12]:
import torchvision.transforms as T
import pandas as pd

# importar a classe
from SupvDset import SupervisedDataset

# definir variaveis
data_path = 'images'
transforms = T.Compose([
    T.Resize((512,512)),
    T.ToTensor(),
])


# aplicar a classe
train_data = SupervisedDataset(data_path, train_df, transforms)
val_data = SupervisedDataset(data_path, val_df, transforms)

converter para dataframe

In [13]:
df_train = {
    "img_id": [],
    #"img": [],
    "img_origin": [],
    "img_placement": [],
    "nr_boil": [],
    "polygons_boil": [],
    "nr_pan": [],
    "polygons_pan": []
}

for i in range(len(train_data)):
    if i % 500 == 0:
        print(f"Progresso: {i}/{len(train_data)}")
    img, _, polygons_pan, polygons_boil, nr_boil, nr_pan, img_id, img_origin, img_placement = train_data[i]
    df_train["img_id"].append(img_id)
    #df_train["img"].append(img)
    df_train["img_origin"].append(img_origin)
    df_train["img_placement"].append(img_placement)
    df_train["nr_boil"].append(nr_boil)
    df_train["polygons_boil"].append(polygons_boil)
    df_train["nr_pan"].append(nr_pan)
    df_train["polygons_pan"].append(polygons_pan)

df_train = pd.DataFrame(df_train)

Progresso: 0/2439
Progresso: 500/2439
Progresso: 1000/2439
Progresso: 1500/2439
Progresso: 2000/2439


In [16]:
df_val = {
    "img_id": [],
    #"img": [],
    "img_origin": [],
    "img_placement": [],
    "nr_boil": [],
    "polygons_boil": [],
    "nr_pan": [],
    "polygons_pan": []
}

for i in range(len(val_data)):
    if i % 500 == 0:
        print(f"Progresso: {i}/{len(val_data)}")
    img, _, polygons_pan, polygons_boil, nr_boil, nr_pan, img_id, img_origin, img_placement = val_data[i]
    df_val["img_id"].append(img_id)
    #df_val["img"].append(img)
    df_val["img_origin"].append(img_origin)
    df_val["img_placement"].append(img_placement)
    df_val["nr_boil"].append(nr_boil)
    df_val["polygons_boil"].append(polygons_boil)
    df_val["nr_pan"].append(nr_pan)
    df_val["polygons_pan"].append(polygons_pan)

df_val = pd.DataFrame(df_val)

Progresso: 0/610
Progresso: 500/610


guardar os datasets

In [17]:
df_train.to_pickle("Model_Train.pkl")
df_val.to_pickle("Model_Val.pkl")

In [18]:
df_train.sample(3)

,img_id,img_origin,img_placement,nr_boil,polygons_boil,nr_pan,polygons_pan
10,IDIxbudJIyNOaARlD,D,roof,[],[],[1],"[[(234.312, 227.44266666666664), (308.04, 224...."
2119,IDP9PuKcINe6ZP,D,roof,[],[],"[2, 2]","[[(12.672, 59.562666666666665), (31.616, 86.52..."
2372,IDnuFOsB9ymgO9ks,D,roof,[],[],"[2, 1]","[[(444.8, 69.56521739130434), (448.87272727272..."


In [19]:
df_val.sample(3)

,img_id,img_origin,img_placement,nr_boil,polygons_boil,nr_pan,polygons_pan
320,IDbVgc3KZM6BbqCOF,D,roof,[],[],"[1, 1, 1, 1, 1, 1, 1]","[[(350.8363636363636, 61.73508594539939), (350..."
525,IDKgqKl5InA,D,roof,[],[],[1],"[[(169.984, 163.84), (218.112, 225.96266666666..."
445,IDW7g3ooNM,D,roof,[],[],[1],"[[(162.944, 196.43733333333333), (190.72, 206...."
